| [**Overview**](./00_overview.ipynb) |  **Examples:** |  [Selecting and Indexing Geochem Data](01_indexes_selectors.ipynb) | [Data Munging](02_munging.ipynb) | [Visualisation](03_visualisation.ipynb) |[lambdas](04_lambdas.ipynb) |
|:-----|:-----|:-----|:-----|:-----|:-----|


## Selecting and Indexing Geochemical Data

The `pyrolite.pyrochem` API provides access to indexing and transformation functions. This allows easy subsetting of geochemical datasets which can otherwise be unweildly (expecially as the number of columns increases..). To provide a simple illustration we generate a synthetic dataset to work from, which contains an array of typical geochemical measures - oxide components, element components (here as ppm), element ratios and isotope ratios. While this size dataset is managable, some of the indexing tools pyrolite provides make it straightforward to pull out different parts of the dataset.

First let's create some data to play with:

In [1]:
import pyrolite.geochem
import pandas as pd
import numpy as np
from pyrolite.util.synthetic import normal_frame

pd.set_option("precision", 3)  # smaller graphical outputs
 
df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']) * 100
df[['Ni', 'Ti', 'La', 'Lu', 'Te']] *= 10
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Sr87/Sr86
0,0.954,23.453,7.353,3.872,14.159,127.588,38.187,32.554,33.789,269.977,0.71
1,0.846,27.894,7.279,4.365,15.610,114.509,30.983,31.776,31.538,231.254,0.71
2,0.879,28.094,6.752,4.190,16.441,122.442,26.854,33.936,32.686,220.517,0.71
3,0.918,22.381,6.233,3.399,16.462,132.042,33.109,33.167,37.957,269.797,0.71
4,0.837,25.481,6.557,3.862,16.273,127.267,32.482,34.650,33.865,241.640,0.71
5,0.818,28.745,6.804,4.112,15.902,117.508,30.204,33.493,31.953,223.024,0.71
6,0.992,24.937,6.758,4.022,18.057,119.914,33.033,33.901,36.176,229.315,0.71
7,1.013,23.011,7.319,4.231,18.305,133.374,34.368,33.624,38.056,221.775,0.71
8,0.945,21.912,6.573,3.416,14.514,133.324,34.214,32.563,34.943,291.361,0.71
9,0.920,24.515,6.955,3.922,15.982,123.703,33.323,32.410,34.168,253.457,0.71


### Selecting Data

In [2]:
df.head(2).pyrochem.oxides

,CaO,MgO,SiO2,FeO,Na2O
0,0.954,23.453,7.353,3.872,14.159
1,0.846,27.894,7.279,4.365,15.610


In [3]:
df.head(2).pyrochem.elements

,Ni,Ti,La,Lu,Te
0,127.588,38.187,32.554,33.789,269.977
1,114.509,30.983,31.776,31.538,231.254


In [4]:
df.head(2).pyrochem.REE

,La,Lu
0,32.554,33.789
1,31.776,31.538


In [5]:
df.head(2).pyrochem.REY

,La,Lu
0,32.554,33.789
1,31.776,31.538


In [6]:
df.head(2).pyrochem.compositional

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te
0,0.954,23.453,7.353,3.872,14.159,127.588,38.187,32.554,33.789,269.977
1,0.846,27.894,7.279,4.365,15.610,114.509,30.983,31.776,31.538,231.254


In [7]:
df.head(2).pyrochem.isotope_ratios

,Sr87/Sr86
0,0.71
1,0.71


------
### Listing Columns
If you just want the column names, it's easy enough to get those too (these are actually used in the indexing above):

In [8]:
df.pyrochem.list_oxides

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O']

In [9]:
df.pyrochem.list_elements

['Ni', 'Ti', 'La', 'Lu', 'Te']

In [10]:
df.pyrochem.list_REE

['La', 'Lu']

In [11]:
df.pyrochem.list_compositional

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']

In [12]:
df.pyrochem.list_isotope_ratios

['Sr87/Sr86']

----
### Using Indexers, Scaling

You can also use these indexers for assignment, where the dimensionality of the dataset doesn't change. While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [13]:
from pyrolite.util.units import scale

els = df.pyrochem.elements
els.pyrochem.elements *= scale('ppm', 'wt%')

In [14]:
df.pyrochem.elements, els.pyrochem.elements

(        Ni      Ti      La      Lu       Te
 0  127.588  38.187  32.554  33.789  269.977
 1  114.509  30.983  31.776  31.538  231.254
 2  122.442  26.854  33.936  32.686  220.517
 3  132.042  33.109  33.167  37.957  269.797
 4  127.267  32.482  34.650  33.865  241.640
 5  117.508  30.204  33.493  31.953  223.024
 6  119.914  33.033  33.901  36.176  229.315
 7  133.374  34.368  33.624  38.056  221.775
 8  133.324  34.214  32.563  34.943  291.361
 9  123.703  33.323  32.410  34.168  253.457,
       Ni     Ti     La     Lu     Te
 0  0.013  0.004  0.003  0.003  0.027
 1  0.011  0.003  0.003  0.003  0.023
 2  0.012  0.003  0.003  0.003  0.022
 3  0.013  0.003  0.003  0.004  0.027
 4  0.013  0.003  0.003  0.003  0.024
 5  0.012  0.003  0.003  0.003  0.022
 6  0.012  0.003  0.003  0.004  0.023
 7  0.013  0.003  0.003  0.004  0.022
 8  0.013  0.003  0.003  0.003  0.029
 9  0.012  0.003  0.003  0.003  0.025)

---
### Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [15]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

,Sr87/Sr86,MgO,SiO2,FeO,Ca,Te,Na,Na/Te,MgO/SiO2
0,0.71,23.453,7.353,3.872,0.682,269.977,10.504,0.039,3.190
1,0.71,27.894,7.279,4.365,0.604,231.254,11.580,0.050,3.832
2,0.71,28.094,6.752,4.190,0.628,220.517,12.197,0.055,4.161
3,0.71,22.381,6.233,3.399,0.656,269.797,12.212,0.045,3.591
4,0.71,25.481,6.557,3.862,0.598,241.640,12.073,0.050,3.886
5,0.71,28.745,6.804,4.112,0.585,223.024,11.797,0.053,4.224
6,0.71,24.937,6.758,4.022,0.709,229.315,13.396,0.058,3.690
7,0.71,23.011,7.319,4.231,0.724,221.775,13.580,0.061,3.144
8,0.71,21.912,6.573,3.416,0.675,291.361,10.767,0.037,3.334
9,0.71,24.515,6.955,3.922,0.657,253.457,11.856,0.047,3.525


In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [16]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

,Sr87/Sr86,FeO,Fe2O3
0,0.71,3.485,0.430
1,0.71,3.929,0.485
2,0.71,3.771,0.466
3,0.71,3.059,0.378
4,0.71,3.476,0.429
5,0.71,3.701,0.457
6,0.71,3.620,0.447
7,0.71,3.808,0.470
8,0.71,3.074,0.380
9,0.71,3.530,0.436
